In [ ]:
# Notebook to present comparison (i.e., "diff") of data for two scenarios for a given set of highway links in map form
# using the hvplot library. The "diff" of total flow, speed, and  V/C, are presented.
# The two sets of input data were previously calculated by sample_highway_links_report.ipypnb, and saved in CSV format.

import numpy as np
import pandas as pd
import geopandas as gp
from io import StringIO
import matplotlib.pyplot as plt
import folium
import bokeh
import geoviews
import xarray as xr
import hvplot.pandas
import hvplot.xarray

In [ ]:
# >>> *** USER INPUT REQUIRED *** <<<
#
# (1) Directory in which user's output CSV report data was saved - it will now be our *input* directory.
#
my_sandbox_dir = r'S:/my_modx_output_dir/'
#
# (2) Name of CSV file in this directory with report for "base" scenario.
#
base_csv_fn = 'links_report_base_scenario.csv'
#
# (3) Name of CSV file in this directory with report for "comparison" scenario.
#
comparison_csv_fn = 'links_report_comparison_scenario.csv'

In [ ]:
# Fully-qualified pathnames to CSV files containing "base" and "comparison" reports
#
fq_base_csv_fn = my_sandbox_dir + base_csv_fn
fq_comparison_csv_fn = my_sandbox_dir + comparison_csv_fn

In [ ]:
# Read "base" data into a dataframe
#
base_df = pd.read_csv(fq_base_csv_fn, delimiter=',')
#
# Save the list of links for which the reports were generated
#
links_list = base_df['ID1'].to_list()
#
# Save column names for (possible) later use, after having removed the "ID1" column at index[0]
#
saved_column_names = base_df.columns[1:]
#
# Maybe we don't need to do the following, thanks to the functionality of df.merge()...
# And rename each of the colums with the prefix 'base_'
# base_df.rename(columns=lambda x: 'base_' + x, inplace=True, )

In [ ]:
# Read "comparison" data into a dataframe
comparison_df = pd.read_csv(fq_comparison_csv_fn, delimiter=',')
# Maybe we don't need to do the following, thanks to the functionality of df.merge()...
# And rename each of the colums with the prefix 'comparison_'
# comparison_df.rename(columns=lambda x: 'comparison_' + x, inplace=True, )

In [ ]:
delta_df = pd.merge(left=comparison_df, right=base_df, on="ID1", suffixes=('_comp', '_base'))

In [ ]:
delta_df

In [ ]:
# Calculate the delta (comparision - base) for each metric
#
for column_name in saved_column_names:
    base_column_name = column_name + '_base'
    comp_column_name = column_name + '_comp'
    delta_column_name = column_name + '_delta'
    delta_df[delta_column_name] = delta_df[comp_column_name] - delta_df[base_column_name]
# end_for

In [ ]:
delta_df

In [ ]:
# Directory in which the spatial data for the model network links is stored (both shapefile and GeoJSON formats)
links_spatial_data_dir = r'G:/Data_Resources/modx/statewide_links_shapefile/'

In [ ]:
# Load the links shapefile into a geopandas dataframe 
# NOTE: This version of the shapefile is in EPSG4326, i.e., WGS84
links_shapefile_fn = 'Statewide_Links_2018_BK_EPSG4326.shp'
fq_links_shapefile_fn = links_spatial_data_dir + links_shapefile_fn
links_gdf = gp.read_file(fq_links_shapefile_fn)
links_gdf.set_index("ID")

In [ ]:
# Filter the links geodataframe to only the links of interest
filtered_links_gdf = links_gdf[links_gdf['ID'].isin(links_list)] 

In [ ]:
filtered_links_gdf

In [ ]:
# Join the geo-data frame for the links with the "links_data_df", which contains the computed data about these links
join_df = filtered_links_gdf.join(delta_df.set_index("ID1"), on="ID")

In [ ]:
join_df

In [ ]:
# Return the bounding box of all the features in a geo-dataframe.
# The bounding box is returned as a dictionary with the following keys: { 'minx', 'miny', 'maxx', 'maxy'}.
#
def bbox_of_gdf(gdf):
    bounds_tuples = gdf['geometry'].map(lambda x: x.bounds)
    bounds_dicts = []
    for t in bounds_tuples:
        temp = { 'minx' : t[0], 'miny' : t[1], 'maxx' : t[2], 'maxy' : t[3] }
        bounds_dicts.append(temp)
    # end_for
    bounds_df = pd.DataFrame(bounds_dicts)
    minx = bounds_df['minx'].min()
    miny = bounds_df['miny'].min()
    maxx = bounds_df['maxx'].max()
    maxy = bounds_df['maxy'].max()
    retval = { 'minx' : minx, 'miny' : miny, 'maxx' : maxx, 'maxy' : maxy }
    return retval
# end_def bbox_of_gdf()

# Given a dictonary of the form  'minx', 'miny', 'maxx', 'maxy'} representing a geographic bounding box,
# return the center point as a dictionary with the keys { 'x' , 'y' }.
def center_of_bbox(bbox):
    center_x = bbox['minx'] + (bbox['maxx'] - bbox['minx']) / 2
    center_y = bbox['miny'] + (bbox['maxy'] - bbox['miny']) / 2
    retval = { 'x' : center_x, 'y' : center_y }
    return retval
# end_def center_of_bbox()

In [ ]:
# Get the bounding box of the selected links, and the center point of that bounding box
bbox = bbox_of_gdf(join_df)
center_pt = center_of_bbox(bbox)

In [ ]:
# Directory containing miscellaneous reference data
misc_reference_data_dir = r'G:/Data_Resources/modx/misc_reference_data/'

In [ ]:
# Load the MassGIS TOWNS_POLYM shapefile into a geopandas dataframe 
# NOTE: This version of the shapefile is in EPSG4326, i.e., WGS84
towns_shapefile_fn = 'towns_polym_EPSG4326.shp'
fq_towns_shapefile_fn = misc_reference_data_dir + towns_shapefile_fn
towns_gdf = gp.read_file(fq_towns_shapefile_fn)
towns_gdf.set_index("town_id")

In [ ]:
# Export the geo-dataframe to GeoJSON format, so it can be used with the folium library
out_geojson_fn = my_sandbox_dir + 'temp_geojson.geojson'
join_df.to_file(out_geojson_fn, driver='GeoJSON')

In [ ]:
# Make a static map of the change in speed during the AM period overlayed on the towns layer
base = towns_gdf.plot(color='white', edgecolor='black')
join_df.plot("Speed_am_delta", ax=base, figsize=(20.0,16.0), cmap='plasma', legend=True)
plt.xlim((bbox['minx'], bbox['maxx']))
plt.ylim((bbox['miny'], bbox['maxy']))
plt.title('Chage in Speed in AM')
plt.show()

In [ ]:
# Render an interactive folium map of the change in AM speed
# 
center = [center_pt['y'], center_pt['x']]
m = folium.Map(location=center, zoom_start=12)
links_geojson = open(out_geojson_fn).read()
#
# Color scale source: https://colorbrewer2.org/#type=diverging&scheme=RdYlGn&n=6 (inverted)
def speed_colorscale(delta_speed):
    if delta_speed == None:
        retval = '#000000'
    elif (delta_speed > 20.0):
        retval = '#1a9850'
    elif (delta_speed > 10.0):
        retval = '#91cf60'
    elif (delta_speed > 0.0):
        retval = '#d9ef8b'
    elif (delta_speed > -10.0):
        retval = '#fee08b'
    elif (delta_speed > -20.0):
        retval = '#fc8d59'
    else:
        retval = '#d73027'
    #
    return retval
#
def my_style_function(feature):
    delta_speed = feature['properties']['Speed_am_delta']
    return {
        'opacity': 1.0,
        'weight' : 5.0,
        'color': speed_colorscale(delta_speed)
    }
#
folium.GeoJson(links_geojson,
               style_function=my_style_function,
               tooltip = folium.GeoJsonTooltip(fields=('ID', 'Tot_Flow_daily_delta', 'Speed_am_delta', 'VOC_am_delta'),
                                               aliases=('Link ID',
                                                        'Change in Daily total flow', 
                                                        'Change in AM speed', 
                                                        'Change in AM V/C'))).add_to(m)

#
m

In [ ]:
# Make an interactive bar chart of the change in speed for each link in the AM period
delta_df.hvplot.barh(x='ID1', xlabel='Link ID', 
                          y='Speed_am_delta', ylabel='Change in Speed (MPH) AM Period', xformatter="%f", height=1500)

In [ ]:
# Make a static map of the change in volume-to-capacity ratio during the AM period overlayed on the towns layer
base = towns_gdf.plot(color='white', edgecolor='black')
join_df.plot("VOC_am_delta", ax=base, figsize=(20.0,16.0), cmap='plasma', legend=True)
plt.xlim((bbox['minx'], bbox['maxx']))
plt.ylim((bbox['miny'], bbox['maxy']))
plt.title('Chage in VOC in AM')
plt.show()

In [ ]:
# Make an interactive bar chart of the change in volume-to-capacity ratio for each link in the AM period
delta_df.hvplot.barh(x='ID1', xlabel='Link ID', 
                          y='VOC_am_delta', ylabel='Change in V/C Ratio - AM Period', xformatter="%f", height=1500)

In [ ]:
# Render an interactive folium map of the change in AM volume-to-capacity ratio
# 
center = [center_pt['y'], center_pt['x']]
m = folium.Map(location=center, zoom_start=12)
links_geojson = open(out_geojson_fn).read()
#
# Color scale source: https://colorbrewer2.org/#type=diverging&scheme=RdYlGn&n=6 (inverted)
def voc_colorscale(delta_voc):
    if delta_voc == None:
        retval = '#000000'
    elif (delta_voc > 0.5):
        retval = '#2171b5'
    elif (delta_voc > 0.25):
        retval = '#6baed6'
    elif (delta_voc > 0.0):
        retval = '#bdd7e7'
    else:
        retval = '#eff3ff'
    #
    return retval
#
def my_style_function(feature):
    delta_voc = feature['properties']['VOC_am_delta']
    return {
        'opacity': 1.0,
        'weight' : 5.0,
        'color': voc_colorscale(delta_voc)
    }
#
folium.GeoJson(links_geojson,
               style_function=my_style_function,
               tooltip = folium.GeoJsonTooltip(fields=('ID', 'Tot_Flow_daily_delta', 'Speed_am_delta', 'VOC_am_delta'),
                                               aliases=('Link ID',
                                                        'Change in Daily total flow', 
                                                        'Change in AM speed', 
                                                        'Change in AM VOC'))).add_to(m)

#
m

In [ ]:
# Make a static map of the change in total daily flow (volume) ratio during the AM period overlayed on the towns layer
base = towns_gdf.plot(color='white', edgecolor='black')
join_df.plot("Tot_Flow_daily_delta", ax=base, figsize=(20.0,16.0), cmap='plasma', legend=True)
plt.xlim((bbox['minx'], bbox['maxx']))
plt.ylim((bbox['miny'], bbox['maxy']))
plt.title('Change in Daily Total Flow (volume)')
plt.show()

In [ ]:
# Make an interactive bar chart of the change in total daily flow (volume) by link
delta_df.hvplot.barh(x='ID1', xlabel='Link ID', 
                          y='Tot_Flow_daily_delta', ylabel='Chagne in Total Daily Volume', xformatter="%f", height=1500)

In [ ]:
# Make an interactive folium map of the change in total daily flow (volume) 
# 
center = [center_pt['y'], center_pt['x']]
m = folium.Map(location=center, zoom_start=12)
links_geojson = open(out_geojson_fn).read()
#
# Colorscale source = https://colorbrewer2.org/#type=sequential&scheme=Reds&n=7 (inverted)
def flow_colorscale(delta_flow):
    if delta_flow == None:
        retval = '#000000'
    elif (delta_flow > 10000.0):
        retval = '#99000d'
    elif (delta_flow > 0.0):
        retval = '#cb181d'
    elif (delta_flow > -10000.0):
        retval = '#ef3b2c'
    elif (delta_flow > -20000.0):
        retval = '#fb6a4a'
    elif (delta_flow > -30000.0):
        retval = '#fc9272'
    elif (delta_flow > -40000.0):
        retval = '#fcbba1'
    else:
        retval = '#fee5d9'
    #
    return retval
#
def my_style_function(feature):
    delta_flow = feature['properties']['Tot_Flow_daily_delta']
    return {
        'opacity': 1.0,
        'weight' : 5.0,
        'color': flow_colorscale(delta_flow)
    }
#
folium.GeoJson(links_geojson,
               style_function=my_style_function,
               tooltip = folium.GeoJsonTooltip(fields=('ID', 'Tot_Flow_daily_delta', 'Speed_am_delta', 'VOC_am_delta'),
                                               aliases=('Link ID',
                                                        'Change in Daily total flow', 
                                                        'Change in AM speed', 
                                                        'Change in AM V/C'))).add_to(m)
#
m